### 000128 - 2D tuning curves demonstration

This notebook demonstrates the calculation of 2D tuning curves using Pynapple and the visualization of the results using Figurl.

To produce the results in this notebook, you will need to initialize your kachery-cloud client via `kachery-cloud-init` in a terminal.

In [1]:
import h5py
import pynwb
import pynapple as nap
import dendro.client as prc
import remfile
from TuningCurves2DView import TuningCurves2DView


# Load project D-000128
project = prc.load_project("a016a71a")
dandiset_id = "000128"

# Select an NWB file
asset_path = "sub-Jenkins/sub-Jenkins_ses-full_desc-train_behavior+ecephys.nwb"

# Lazy load NWB file
file = remfile.File(project.get_file(f"imported/{dandiset_id}/{asset_path}"))
io = pynwb.NWBHDF5IO(file=h5py.File(file, "r"), mode="r")
nwbfile = io.read()

In [2]:
# Load the cursor position into a pynapple TsdFrame
spatial_series = nwbfile.processing["behavior"]["cursor_pos"]
position_over_time = nap.TsdFrame(
    d=spatial_series.data[:],
    t=spatial_series.timestamps[:],
    columns=["x", "y"],
)

In [3]:
# Load the unit spike times into a pynapple TsGroup
unit_names = nwbfile.units["id"][:]
unit_spike_times = nwbfile.units["spike_times"][:]
spike_times_group = nap.TsGroup(
    {i: unit_spike_times[i] for i in range(len(unit_names))}
)

/tmp/ipykernel_101884/2119542534.py:4: UserWarning: Elements should not be passed as numpy array. Default time units is seconds when creating the Ts object.
  spike_times_group = nap.TsGroup(


In [4]:
# Compute 2D tuning curves
num_bins = 30
rate_maps, position_bins = nap.compute_2d_tuning_curves(
    spike_times_group,
    position_over_time,
    num_bins,
)

/home/magland/miniconda3/envs/D-000128/lib/python3.9/site-packages/pynapple/process/tuning_curves.py:203: RuntimeWarning: invalid value encountered in divide
  count = count / occupancy


In [5]:
# Construct a tuning curves figurl view
V = TuningCurves2DView(
    rate_maps=[rate_maps[i] for i in range(len(unit_names))],
    x_bin_positions=position_bins[0],
    y_bin_positions=position_bins[1],
    unit_ids=unit_names,
    unit_num_spikes=[len(spike_times_group[i]) for i in range(len(unit_names))],
)
view_url = V.get_view_url(label=f"{dandiset_id}/{asset_path}|tuning-curves-2d")

# Print the view URL
print(view_url)

# Display the view in an iframe
from IPython.display import IFrame
display(IFrame(view_url + "&hide=1", width="95%", height=500))

https://figurl.org/f?v=https://figurl-tuning-curves-1.surge.sh&d=sha1://2abecf4d2710010de0067a7fe7c9ecb237fb9bf3&label=000128/sub-Jenkins/sub-Jenkins_ses-full_desc-train_behavior+ecephys.nwb|tuning-curves-2d
